# PGA Tour Web Scraping
### *Data is obtained using PGATour.com's GraphQL API*


Initially, when I attempted to scrape data from pgatour.com using traditional web scraping techniques with BeautifulSoup, I encountered a roadblock. It became evident that pgatour.com utilizes GraphQL, a query language for APIs, to handle its data. This meant that directly scraping the website's HTML structure wouldn't provide the desired results. Realizing the need to adapt, I learned about GraphQL and how to effectively query the data using python. With this knowledge and understanding, I implemented GraphQL queries to retrieve the specific data I required from pgatour.com. This transition allowed me to efficiently extract the desired information from the website, enabling me to proceed with my data analysis and exploration.


### Tasks
This Jupyter Notebook contains code for retrieving and analyzing PGA Tour data using the GraphQL API. The code performs the following tasks:

1. Imports the necessary packages and sets up the API endpoint.
2. Sends a GraphQL introspection query to retrieve the schema.
3. Saves the GraphQL schema as a JSON file.
4. Defines functions to retrieve player statistics, player information, and tournament winners.
5. Retrieves player statistics and information for multiple years.
6. Merges the data into a single DataFrame.
7. Performs data cleaning and type conversion.
8. Saves the final DataFrame as a CSV file.

### Code Explanation

The code is divided into several sections:

1. Importing Packages: Importing the required packages, including `requests`, `json`, and `pandas`.

2. API Configuration: Defining the API endpoint URL and the `X_API_KEY` token.

3. GraphQL Introspection Query: Defining the GraphQL introspection query to retrieve the schema.

4. Retrieving the Schema: Sending the introspection query to the server and saving the schema as a JSON file.

5. Function Definitions: Defining helper functions to retrieve player statistics, player information, and tournament winners.

6. Data Retrieval and Processing: Looping through the desired years and retrieving player statistics, player information, and tournament winners. Merging the data into a single DataFrame.

7. Data Cleaning and Type Conversion: Performing data cleaning and converting columns to their appropriate data types.

8. Saving the Data: Saving the final DataFrame as a CSV file.




In [1]:
##### Import all required packages #####
import requests
#import graphql
import json
from numpy import NaN
import pandas as pd


In [2]:
# in the requests header seems to be a constant token ('x-api-key') that is needed
# found in devtools
X_API_KEY = "da2-gsrx5bibzbb4njvhl7t37wqyl4"

url = "https://orchestrator.pgatour.com/graphql"

# GraphQL introspection query
introspection_query = """
    query IntrospectionQuery {
        __schema {
            queryType {
                name
            }
            mutationType {
                name
            }
            subscriptionType {
                name
            }
            types {
                ...FullType
            }
            directives {
                name
                description
                locations
                args {
                    ...InputValue
                }
            }
        }
    }

    fragment FullType on __Type {
        kind
        name
        description
        fields(includeDeprecated: true) {
            name
            description
            args {
                ...InputValue
            }
            type {
                ...TypeRef
            }
            isDeprecated
            deprecationReason
        }
        inputFields {
            ...InputValue
        }
        interfaces {
            ...TypeRef
        }
        enumValues(includeDeprecated: true) {
            name
            description
            isDeprecated
            deprecationReason
        }
        possibleTypes {
            ...TypeRef
        }
    }

    fragment InputValue on __InputValue {
        name
        description
        type {
            ...TypeRef
        }
        defaultValue
    }

    fragment TypeRef on __Type {
        kind
        name
        ofType {
            kind
            name
            ofType {
                kind
                name
                ofType {
                    kind
                    name
                    ofType {
                        kind
                        name
                        ofType {
                            kind
                            name
                            ofType {
                                kind
                                name
                            }
                        }
                    }
                }
            }
        }
    }
"""

# Send the introspection query to the server
response = requests.post(
    url, json={"query": introspection_query}, headers={"x-api-key": X_API_KEY}
)

# Print the response (GraphQL schema)
json_schema = response.json()

with open("graphql_schema.json", "w") as f:
    # Use the `json.dump()` method to write the data to the file
    json.dump(json_schema, f, indent=4)

In [3]:
# Stat IDs and Stat Description used for queries
# This will be the features in final data set
stats = {
    "120": "ScoringAvg",
    "156": "BirdieAvg",
    "101": "DrivingDistance",
    "129": "TotalDriving",
    "103": "GIR%",
    "130": "Scrambling",
    "02675": "SG_Total",
    "02568": "SG_ApproachGreen",
    "02564": "SG_Putting",
    "02567": "SG_OffTee",
    "02569": "SG_AroundGreen",
    "02674": "SG_TeeToGreen",
    "02394": "FedExCupPoints",
    "138": "Top10Finishes",
    "186": "WorldRank",
    "109": "Money"
}

In [4]:
# some functions containing queries used to extract data from PGATour.com
# Data Structures varied, therefore multiple functions/queries were needed

def get_df(YEAR, STAT_ID, DESCR):
    X_API_KEY = "da2-gsrx5bibzbb4njvhl7t37wqyl4"
    req = {
        "operationName": "StatDetails",
        "variables": {
            "tourCode": "R",
            "statId": STAT_ID,
            "year": YEAR,
            "eventQuery": None,
        },
        "query": """query StatDetails($tourCode: TourCode!, $statId: String!, $year: Int, $eventQuery: StatDetailEventQuery) {
  statDetails(
    tourCode: $tourCode
    statId: $statId
    year: $year
    eventQuery: $eventQuery
  ) {
    tourCode
    year
    displaySeason
    statId
    statType
    tournamentPills {
      tournamentId
      displayName
    }
    yearPills {
      year
      displaySeason
    }
    statTitle
    statDescription
    tourAvg
    lastProcessed
    statHeaders
    statCategories {
      category
      displayName
      subCategories {
        displayName
        stats {
          statId
          statTitle
        }
      }
    }
    rows {
      ... on StatDetailsPlayer {
        __typename
        playerId
        playerName
        country
        countryFlag
        rank
        rankDiff
        rankChangeTendency
        stats {
          statName
          statValue
          color
        }
      }
      ... on StatDetailTourAvg {
        __typename
        displayName
        value
      }
    }
    sponsorLogo
    }
    } 
    """,
    }

    # post the request
    page = requests.post(
        url,
        json=req,
        headers={"x-api-key": X_API_KEY},
    )
    # get the data
    data = page.json()["data"]["statDetails"]["rows"]

    # filter out items, thats __typename is not "StatDetailsPlayer" like
    # data contains -> "__typename": "StatDetailTourAvg", remove the tour average value
    data = filter(lambda item: item.get("__typename", NaN) == "StatDetailsPlayer", data)

    # format to a table that is in the webpage
    
    table = map(
        lambda item: {
            # "RANK": item["rank"],
            "PID" : item["playerId"],
            "PLAYER": item["playerName"],
            DESCR: item["stats"][0]["statValue"],
        },
        data,
    )

    # convert the dataframe
    s = pd.DataFrame(table)

    return s

def get_players():
    url = "https://orchestrator.pgatour.com/graphql"

    X_API_KEY = "da2-gsrx5bibzbb4njvhl7t37wqyl4"
    req = {
    "operationName": "PlayerDirectory",
    "variables": {
        "tourCode": "R"
    },
    "query": """query PlayerDirectory($tourCode: TourCode!, $active: Boolean) {  playerDirectory(tourCode: $tourCode, active: $active) {    tourCode    players {      id      isActive      firstName      lastName      shortName      displayName      alphaSort      country      countryFlag      headshot      playerBio {        id        age        education        turnedPro      }    }  }}
    """,
    }

    # post the request
    page = requests.post(
    url,
    json=req,
    headers={"x-api-key": X_API_KEY},
    )
    # get the data
    # print(page.json())
    data = page.json()["data"]["playerDirectory"]["players"]

    # filter out items, thats __typename is not "StatDetailsPlayer" like
    # data contains -> "__typename": "StatDetailTourAvg", remove the tour average value
    # data = filter(lambda item: item.get("__typename", NaN) == "StatDetailsPlayer", data)

    # format to a table that is in the webpage

    table = map(
    lambda item: {
        # "RANK": item["rank"],
        "PID" : item["id"],
        "PLAYER": item['displayName'],
        "Country": item["countryFlag"],
    },
    data,
    )

    # convert the dataframe
    s = pd.DataFrame(table)

    return s

def get_wins(YEAR):
    X_API_KEY = "da2-gsrx5bibzbb4njvhl7t37wqyl4"
    req = {
    "operationName": "TourCupSplit",
    "variables": {
        "id": "fedex",
        "tourCode": "R",
        "year": YEAR
    },
    "query": """query TourCupSplit($tourCode: TourCode!, $id: String, $year: Int, $eventQuery: StatDetailEventQuery) {
      tourCupSplit(tourCode: $tourCode, id: $id, year: $year, eventQuery: $eventQuery) {
        id
        title
        projectedTitle
        projectedLive
        season
        description
        detailCopy
        logo
        options
        fixedHeaders
        columnHeaders
        rankingsHeader
        message
        projectedPlayers {
          ...Player
          ...InfoRow
        }
        officialPlayers {
          ...Player
          ...InfoRow
        }
        tournamentPills {
          tournamentId
          displayName
        }
        yearPills {
          year
          displaySeason
        }
        winner {
          id
          rank
          firstName
          lastName
          displayName
          shortName
          countryFlag
          country
          earnings
          totals {
            label
            value
          }
        }
      }
    }

    fragment Player on TourCupCombinedPlayer {
      __typename
      id
      firstName
      lastName
      displayName
      shortName
      countryFlag
      country
      rankingData {
        projected
        official
        event
        movement
        movementAmount
        logo
        logoDark
      }
      pointData {
        projected
        official
        event
        movement
        movementAmount
        logo
        logoDark
      }
      projectedSort
      officialSort
      thisWeekRank
      previousWeekRank
      columnData
    }

    fragment InfoRow on TourCupCombinedInfo {
      logo
      logoDark
      text
      sortValue
    }
    """,
    }

    # post the request
    page = requests.post(
    url,
    json=req,
    headers={"x-api-key": X_API_KEY},
    )
    # get the data
    #print(page.json())
    data = page.json()["data"]["tourCupSplit"]["officialPlayers"]

    # filter out items, thats __typename is not "StatDetailsPlayer" like
    # data contains -> "__typename": "StatDetailTourAvg", remove the tour average value
    # data = filter(lambda item: item.get("__typename", NaN) == "StatDetailsPlayer", data)

    # format to a table that is in the webpage

    table = map(
    lambda item: {
        # "RANK": item["rank"],
        "PID" : item["id"],
        "PLAYER": item["displayName"],
        "#Wins": item["columnData"][2]
    },
    data,
    )

    # convert the dataframe
    s = pd.DataFrame(table)

    return s

In [5]:
# Loop through each year and continually merge the data frame to store all data from 2007-2022
years = [year for year in range(2007, 2023)]
dfs = []

players = get_players()
#print(players.head())
for year in years:
    df = pd.DataFrame()
    for key in stats:
        if len(df) == 0:
            df = get_df(year, key, stats[key])
            #print(df)
        else:
            curr = get_df(year, key, stats[key])
            #print(df)
            #print(curr)
            df = pd.merge(df, curr, on=["PID","PLAYER"])
    df['Year'] = year  # Add a 'year' column with the current year
    wins = get_wins(year)

    df = pd.merge(df, players, on=["PID","PLAYER"])
    df = pd.merge(df,wins,on=["PID","PLAYER"])

    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)

combined_df.head()

,PID,PLAYER,ScoringAvg,BirdieAvg,DrivingDistance,TotalDriving,GIR%,Scrambling,SG_Total,SG_ApproachGreen,...,SG_OffTee,SG_AroundGreen,SG_TeeToGreen,FedExCupPoints,Top10Finishes,WorldRank,Money,Year,Country,#Wins
0,08793,Tiger Woods,67.794,4.03,302.4,164,71.02,61.02,3.092,1.653,...,.624,.103,2.380,"30,574",12,20.40,"$10,867,052",2007,USA,2
1,06522,Ernie Els,69.294,3.64,297.9,210,64.66,60.16,1.578,.994,...,.452,.163,1.609,"8,980",5,6.60,"$2,705,715",2007,RSA,-
2,22405,Justin Rose,69.301,3.81,288.7,191,65.73,59.89,.878,.404,...,.157,.255,.816,"8,698",7,6.02,"$2,705,875",2007,ENG,-
3,06527,Steve Stricker,69.394,3.61,283.7,243,65.83,60.98,1.486,.811,...,.002,.351,1.164,"10,621",9,6.65,"$4,663,077",2007,USA,1
4,01810,Phil Mickelson,69.456,3.70,298.1,210,64.95,59.15,1.597,.614,...,.328,.499,1.441,"16,037",7,9.04,"$5,819,988",2007,USA,1


In [6]:
# Change empty values to a 0, this will allow conversion to integer
combined_df["#Wins"] = combined_df["#Wins"].replace("-",0)
combined_df.head()

,PID,PLAYER,ScoringAvg,BirdieAvg,DrivingDistance,TotalDriving,GIR%,Scrambling,SG_Total,SG_ApproachGreen,...,SG_OffTee,SG_AroundGreen,SG_TeeToGreen,FedExCupPoints,Top10Finishes,WorldRank,Money,Year,Country,#Wins
0,08793,Tiger Woods,67.794,4.03,302.4,164,71.02,61.02,3.092,1.653,...,.624,.103,2.380,"30,574",12,20.40,"$10,867,052",2007,USA,2
1,06522,Ernie Els,69.294,3.64,297.9,210,64.66,60.16,1.578,.994,...,.452,.163,1.609,"8,980",5,6.60,"$2,705,715",2007,RSA,0
2,22405,Justin Rose,69.301,3.81,288.7,191,65.73,59.89,.878,.404,...,.157,.255,.816,"8,698",7,6.02,"$2,705,875",2007,ENG,0
3,06527,Steve Stricker,69.394,3.61,283.7,243,65.83,60.98,1.486,.811,...,.002,.351,1.164,"10,621",9,6.65,"$4,663,077",2007,USA,1
4,01810,Phil Mickelson,69.456,3.70,298.1,210,64.95,59.15,1.597,.614,...,.328,.499,1.441,"16,037",7,9.04,"$5,819,988",2007,USA,1


In [7]:
# Update all data types

combined_df['PLAYER'] = combined_df['PLAYER'].astype(object)
combined_df['ScoringAvg'] = combined_df['ScoringAvg'].astype(float)
combined_df['BirdieAvg'] = combined_df['BirdieAvg'].astype(float)
combined_df['DrivingDistance'] = combined_df['DrivingDistance'].astype(float)
combined_df['TotalDriving'] = combined_df['TotalDriving'].astype(float)
combined_df['GIR%'] = combined_df['GIR%'].astype(float)
combined_df['Scrambling'] = combined_df['Scrambling'].astype(float)
combined_df['SG_Total'] = combined_df['SG_Total'].astype(float)
combined_df['SG_ApproachGreen'] = combined_df['SG_ApproachGreen'].astype(float)
combined_df['SG_Putting'] = combined_df['SG_Putting'].astype(float)
combined_df['SG_OffTee'] = combined_df['SG_OffTee'].astype(float)
combined_df['SG_AroundGreen'] = combined_df['SG_AroundGreen'].astype(float)
combined_df['SG_TeeToGreen'] = combined_df['SG_TeeToGreen'].astype(float)
combined_df['FedExCupPoints'] = combined_df['FedExCupPoints'].str.replace(',', '',regex=True).astype(int)
combined_df['Top10Finishes'] = combined_df['Top10Finishes'].astype(int)
combined_df['WorldRank'] = combined_df['WorldRank'].astype(float)
combined_df['Money'] = combined_df['Money'].str.replace('[$,]', '',regex=True).astype(int)
combined_df['#Wins'] = combined_df['#Wins'].astype(int)

combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2353 entries, 0 to 2352
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PID               2353 non-null   object 
 1   PLAYER            2353 non-null   object 
 2   ScoringAvg        2353 non-null   float64
 3   BirdieAvg         2353 non-null   float64
 4   DrivingDistance   2353 non-null   float64
 5   TotalDriving      2353 non-null   float64
 6   GIR%              2353 non-null   float64
 7   Scrambling        2353 non-null   float64
 8   SG_Total          2353 non-null   float64
 9   SG_ApproachGreen  2353 non-null   float64
 10  SG_Putting        2353 non-null   float64
 11  SG_OffTee         2353 non-null   float64
 12  SG_AroundGreen    2353 non-null   float64
 13  SG_TeeToGreen     2353 non-null   float64
 14  FedExCupPoints    2353 non-null   int64  
 15  Top10Finishes     2353 non-null   int64  
 16  WorldRank         2353 non-null   float64


In [8]:
# Reorder the columns to make more sense
order = ['PLAYER','Country','Year','ScoringAvg','BirdieAvg','DrivingDistance','TotalDriving',
         'GIR%','Scrambling','SG_Total','SG_ApproachGreen','SG_Putting','SG_OffTee','SG_AroundGreen',
         'SG_TeeToGreen','Top10Finishes','#Wins','FedExCupPoints','WorldRank','Money']
combined_df = combined_df[order]
combined_df.head()

,PLAYER,Country,Year,ScoringAvg,BirdieAvg,DrivingDistance,TotalDriving,GIR%,Scrambling,SG_Total,SG_ApproachGreen,SG_Putting,SG_OffTee,SG_AroundGreen,SG_TeeToGreen,Top10Finishes,#Wins,FedExCupPoints,WorldRank,Money
0,Tiger Woods,USA,2007,67.794,4.03,302.4,164.0,71.02,61.02,3.092,1.653,0.712,0.624,0.103,2.380,12,2,30574,20.40,10867052
1,Ernie Els,RSA,2007,69.294,3.64,297.9,210.0,64.66,60.16,1.578,0.994,-0.031,0.452,0.163,1.609,5,0,8980,6.60,2705715
2,Justin Rose,ENG,2007,69.301,3.81,288.7,191.0,65.73,59.89,0.878,0.404,0.061,0.157,0.255,0.816,7,0,8698,6.02,2705875
3,Steve Stricker,USA,2007,69.394,3.61,283.7,243.0,65.83,60.98,1.486,0.811,0.322,0.002,0.351,1.164,9,1,10621,6.65,4663077
4,Phil Mickelson,USA,2007,69.456,3.70,298.1,210.0,64.95,59.15,1.597,0.614,0.156,0.328,0.499,1.441,7,1,16037,9.04,5819988


In [9]:
# Save the final data frame to a csv to use later

path = 'pga_full.csv'

combined_df.to_csv(path,index=False)

In [10]:
duplicate_rows = df.duplicated(subset=["PLAYER", "Country", "Year"], keep=False).any()
print(duplicate_rows)

False


## Data Dictionary:
Here is a brief description of the columns in the dataset:

| Column Name         | Description                                                                 |
|---------------------|-----------------------------------------------------------------------------|
| PLAYER              | The name of the player.                                                     |
| Country             | The player's home country                                                   |
| Year                | The year in which the data is recorded.                                      |
| ScoringAvg          | The average score of the player.                                             |
| BirdieAvg           | The average number of birdies made by the player per round.                            |
| DrivingDistance     | The average distance of the player's drives.                                 |
| TotalDriving        | The player's total driving score.                                         |
| GIR%                | The percentage of greens in regulation (GIR) achieved by the player.         |
| Scrambling          | The player's scrambling rate, indicating the ability to recover from difficult positions.|
| SG_Total            | The player's strokes gained total.                                           |
| SG_ApproachGreen    | The player's strokes gained on approach shots to the green.                  |
| SG_Putting          | The player's strokes gained through putting.                                 |
| SG_OffTee           | The player's strokes gained off the tee.                                     |
| SG_AroundGreen      | The player's strokes gained around the green.                                |
| SG_TeeToGreen       | The player's strokes gained from tee to green.                               |
| Top10Finishes       | The number of top 10 finishes achieved by the player.                        |
| #Wins               | The number of wins achieved by the player.                                   |
| FedExCupPoints      | The player's total FedEx Cup points.                                         |
| WorldRank           | The player's average points, where higher points indicate a higher world rank.|
| Money               | The amount of money earned by the player.                                    |